In [1]:
from aimon import AnalyzeEval, AnalyzeProd, Application, Model

In [2]:
import os
openai_api_key = os.getenv("OPEN_AI_KEY")

In [3]:
analyze_eval = AnalyzeEval(
    Application("app_jun18_2024"),
    Model("my_gpt4_model_fine_tuned", "GPT-4"), 
    api_key=os.getenv("AIMON_API_KEY"),
    evaluation_name="simple_eval",
    dataset_collection_name="my_first_dataset_collection241"
)

In [4]:
analyze_prod = AnalyzeProd(
    Application("llm_marketing_summarization_app"), 
    Model("my_gpt4_model_v2", "GPT-4"), 
    values_returned=["context", "generated_text"],
    api_key=os.getenv("AIMON_API_KEY")
)

### Evaluation

We will run an evaluation the example below that uses Langchain to summarize documents using OpenAI.

In [5]:
# Lanchain app example
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain

# The analyze_eval decorator will automatically stream through
# records in the specified data collection and run it against 
# this function. The eval_obj is a required parameter that will be
# popular by the analyze_eval decorator
@analyze_eval
def run_application_eval_mode(context_docs=None, user_query=None, prompt=None, my_arg=None):
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(context_docs)
    
    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]
    
    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    return chain.run(docs)

In [6]:
aimon_eval_res = run_application_eval_mode()

/Users/preetamjoshi/projects/aimon/pj_aimon_rely/examples/mflow/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/preetamjoshi/projects/aimon/pj_aimon_rely/examples/mflow/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [7]:
print(aimon_eval_res)

[(' Acme recently launched version 2.1 of their Python library, which has deep integrations with the Python ecosystem and has been proven to be beneficial for developers. This new version includes features like async support and improved error handling.', AnalyzeCreateResponse(message='Data successfully sent to AIMon.')), ('\n\nTo configure the Acme python client, follow the official documentation which includes setting up environment variables and installing dependencies for both basic and advanced setups.', AnalyzeCreateResponse(message='Data successfully sent to AIMon.')), (' The Acme python client is compatible with Python 3.6+ and multiple databases, including MySQL, PostgreSQL, and MongoDB. It is also suitable for cross-language projects with Node.js.', AnalyzeCreateResponse(message='Data successfully sent to AIMon.')), (' The Acme python client may have installation, package conflicts, and connectivity issues. Troubleshooting involves checking the Python environment, dependencie

### Production

We will monitor the example below that uses Langchain to summarize documents using OpenAI.

In [8]:
# Lanchain app example
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain

@analyze_prod
def run_application(context_docs=None, user_query=None, prompt=None, my_arg=None):
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(context_docs)
    
    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]
    
    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    return context_docs, chain.run(docs)

In [9]:
source_text = """
Large Language Models (LLMs) have become integral to automating and enhancing various business processes. 
However, a significant challenge these models face is the concept of \"hallucinations\" - outputs that, 
although fluent and confident, are factually incorrect or nonsensical. For enterprises relying on AI 
for decision-making, content creation, or customer service, these hallucinations can undermine credibility, 
spread misinformation, and disrupt operations. Recently, AirCanada lost a court case due to hallucinations 
in its chatbot [7]. Also, the 2024 Edelman Trust Barometer reported a drop in trust in AI companies from 
61% to 53% compared to 90% 8 years ago [8]. Recognizing the urgency of the issue, we have developed a 
state-of-the-art system designed for both offline and online detection of hallucinations, ensuring higher 
reliability and trustworthiness in LLM outputs.
"""

In [11]:
context, res, aimon_res = run_application(source_text, prompt="Langhchain based summarization of documents")

In [12]:
print(aimon_res)

AnalyzeCreateResponse(message='Data successfully sent to AIMon.')
